# Summary

The main focus of this project is to demonstrate technical proficiency is SQL. We'll be analyzing various Spotify datasets related to artists, albums, and songs to determine which qualitative and quantitative characteristics contribute to a song's popularity. These queries will be used in a subsequent project which will be completed in Tableau and saved to my portfolio.

# Ask

## 1\. Business Task

Identify how various characteristics correlate to a song's popularity score to recommend the formula for a popular song.

# Prepare

## 1\. Dataset Overview

The data source used in this case study was **'Spotify and Genius Track Dataset'** on Kaggle. This dataset was made available by Kaggler Saurabh Shahane.

## 2\. Privacy

Verifying the metadata of our dataset we can confirm it is open-source. The owner has dedicated the work to the public domain by waiving all of his or her rights to the work worldwide under copyright law, including all related and neighboring rights, to the extent allowed by law. You can copy, modify, distribute and perform the work, even for commercial purposes, all without asking permission.

## 3\. Dataset Information

Datasets were curated from Spotify and Genius' respective APIs. The datasets contain varying amounts of information about song, album, and artist features such as but not limited to "liveness", "danceability", BPM, and popularity. The dataset does not contain information about songs, albums, or artists after 2019.

## 4\. Data Organization

Included in the original dataset are 5 seperate datasets divided into 2 folders. The first folder contains 3 datasets - 1 contains both qualitative and quantitative information about artists, 1 about albums, and 1 about songs. The second folder contains 2 datasets - 1 involving lyric data and 1 involving Spotify metrics. For the purpose of this analysis, we will only be working with the first 3 datasets (referred to in this analysis as **artists**, **albums**, and **trackz**).

| Table Name | Type | Description |
| :-- | :-: | --: |
| artists | CSV | artist\_popularity, followers, genres, id, name, track\_id, track\_name\_prev, type |
| albums | CSV | album\_type, artist\_id, available\_markets, external\_urls, href, id, images, name, release\_date |
| trackz | CSV | acousticness, album\_id, analysis\_url, artists\_id, available\_markets, country, danceability, disc\_number, duration\_ms |

## 5\. Data Credibility

Certain attributes were either ambiguous or contained irreconcilable inconsistencies and/or errors and therefore were excluded from our analysis. This included but was not limited to attributes such as **available\_markets**, **country**, **disc\_number**, and others.

  

A limited number of assumptions were made for the sake of consistency throughout this analysis. These assumptions are documented throughout as they occur.

# Process

## 1\. Importing Datasets into Azure Data Studio 2

Datasets were imported into Azure Data Studio 2 to clean and analyze. This process was accomplished through the Import Wizard Azure Data Studio. Tables were assigned the names dbo.artists (for artists), dbo.albums (for albums), and dbo.trackz (for songs).

After importing, a query was run to preview the top 5 rows of each table so we could get an idea of what type of data we're working with and get an idea of any commonalities or discrepencies that might exist. It looks like each track contains a unique ID as does each album and artist.

In [ ]:
/* Viewing 3 tables before joining */

SELECT TOP 5
    *
FROM MusicProj..albums

SELECT TOP 5
    *
FROM
    MusicProj..artists

SELECT TOP 5
    *
FROM
    MusicProj..trackz

## 2\. Cleaning and Manipulating Our Data

### **2.1 Standardizing Artist ID Format**

**(SUBSTR, LEN, LEFT)**

One thing we notice when examining our tables is that, unlike the other tables, the artist\_id attribute in the trackz table is formatted in between sets of brackets and single quotes. To standardize the artist\_id between the table, we'll remove the single quotes and brackets were removed using the SUBSTRING and LEN functions. 

Additionally, many records included multiple values in the artist\_id column. For the sake of this analysis, it's assumed that the first artist\_id listed is the primary album artist while the remaining values are "featured" artists. To ensure that all records only aligned to 1 artist, the LEFT function is used to keep only the primary artist\_id.

In [ ]:
/* Using SUBSTRING, LEN, and LEFT to remove extra characters and multiple artist IDs */

-- Removing first 2 characters using SUBSTRING and LEN

SELECT
    SUBSTRING(
        "artists_id",3,LEN("artists_id"))
FROM 
    MusicProj..trackz

UPDATE 
    MusicProj..trackz
SET 
    artists_id = 
    SUBSTRING(
        "artists_id",3,LEN("artists_id"))

-- Keeping only the primary artist ID using LEFT function

SELECT 
    LEFT(artists_id, 22)
FROM 
    MusicProj..trackz

UPDATE 
    MusicProj..trackz
SET 
    artists_id = LEFT(artists_id, 22)

### **2.2.1 Removing Duplicate Records**

**(CTE, PARTITION BY, ORDER BY, DELETE)**

Since certain tracks can appear on multiple different albums, duplicate records need to be removed to protect the integrity of our data. 

It's assumed that the record with the highest value in the popularity column is the "primary" release version and is therefore be the record used in our sample. The lower popularity duplicate records are removed by partitioning records with the respective row counts and sorting in descending order by popularity score - all within a CTE. Any records with a row count rank higher than 1 are deleted\*.

\*Deleted for the purpose of demonstrating knowledge

In [ ]:
/* Using CTE and Partition By to delete duplicate track records and keep record with highest popularity score */

WITH 
    DupeCTE AS(
    SELECT *, 
    ROW_NUMBER() OVER (
        PARTITION BY 
        duration_ms,  
        tempo, 
        energy, 
        liveness, 
        [name] 
    ORDER BY 
        popularity DESC
        ) row_num
FROM MusicProj..trackz
)

DELETE 
FROM 
    DupeCTE
WHERE 
    row_num > 1

### **2.2.2 Verifying No Duplicates in Remaining Tables**

**(CTE, ROW\_NUMBER, PARTITION BY)**

With duplicates removed from the tracks table, we need to verify that there are no duplicate records for artists or albums on their respective tables. We do this using the same CTE process as described above. The result is 0 records with a row\_num greater that 1 which indicates no duplicates exist.

In [ ]:
/* Verifying no duplicate artists */

WITH 
    artCTE AS(
    SELECT *, 
    ROW_NUMBER() OVER (
        PARTITION BY 
        artists.id,
        [name] 
    ORDER BY
        artist_popularity
        ) row_num
FROM MusicProj..artists
)

SELECT *
FROM 
    artCTE
WHERE 
    row_num > 1

/* Verifying no duplicate albums */

WITH 
    albCTE AS(
    SELECT *, 
    ROW_NUMBER() OVER (
        PARTITION BY 
        albums.id,
        [name] 
    ORDER BY
        release_date
        ) row_num
FROM MusicProj..albums
)

SELECT *
FROM 
    albCTE
WHERE 
    row_num > 1

### **2.3 Standardizing Date Format (YYYY)**

**(LEFT)**

Certain records contain full dates while others only contained years. To standardize these records, all dates were converted to a YYYY format by using the LEFT function to select the first 4 numbers on the left of each record.

In [ ]:
/* Using LEFT to standardize release dates to YYYY format */

UPDATE 
    MusicProj..albums
SET 
    release_date = LEFT(release_date, 4)


### **2.4 Creating Table of Translated Values to Reference in JOIN**

**(CREATE TABLE, INSERT INTO)**

The dataset represents musical key as an integer ranging 0-11 and mode as a 1/0 boolean. 

In order to perform an effective analysis that our stakeholders will comprehend, these attributes need to be translated to string. 

Since C is the first note in the traditional western musical alphabet and said alphabet consists of 12 total notes (7 natural notes and 5 accidentals ranging for A-G), it can logically be assumed that the 0-11 range directly translates to C-B with C = 0 and B = 11.

It's also noted on Spotify's API website that the 1 in the dataset represents major keys while the 0 in the dataset translates to minor. 

In order to translate and include these values in our analysis, two tables were created representing the direct translation. These tables will be joined and referenced in later queries.

In [ ]:
/* Using CREATE and INSERT clauses to create/populate tables to convert key and mode attributes from int to string */

-- Creating tables 

CREATE TABLE 
    MusicProj..keystrings
    (
    key_num int,
    key_let nvarchar(10)
    )

CREATE TABLE 
    MusicProj..modestring
    (
    mode_num int,
    mode_let nvarchar(10)
    )

-- Inserting values

INSERT INTO MusicProj..modestring
VALUES (1,'Maj'),
(0,'min')

INSERT INTO
    MusicProj..keystrings 
    VALUES 
    (0,'C'),
    (1,'C#Db'),
    (2,'D'),
    (3,'D#Eb'),
    (4,'E'),
    (5,'F'),
    (6,'F#Gb'),
    (7,'G'),
    (8,'G#Ab'),
    (9,'A'),
    (10,'A#Bb'),
    (11,'B')

-- Previewing and verifying

SELECT
    *
FROM
    MusicProj..keystrings

SELECT
    *
FROM
    MusicProj..modestring


### **2.5 Joining and Aliasing**

**(LEFT JOIN, INNER JOIN, AS)** 

4 tables are joined to the trackz table to populate remaining relevant info. 2 tables are the modestring and keystrings tables that were created in the previous step for the purpose of converting our key and mode values. The other 2 tables are artists and albums which contain information such as release date, album name, and artist name that can be linked to the individual tracks.

In [ ]:
/* Joining and aliasing 5 tables (tracks, artists, albums, keystrings, modestring) for analysis */
-- First 2 joins convert key and mode attributes from int to string (to be combined as full_key in next step)

SELECT
    *
FROM
    MusicProj..trackz tra
    JOIN 
        MusicProj..keystrings kst
        ON tra.[key] = kst.key_num
    JOIN
        MusicProj..modestring mst
        ON tra.mode = mst.mode_num
    LEFT JOIN
        MusicProj..artists art
        ON tra.artists_id = art.id
    LEFT JOIN 
        MusicProj..albums alb
        ON tra.album_id = alb.id

### **2.6 Combining 2 Columns Into 1**

**(CONCAT, AVG, PARTITION BY)**

Since key and mode are often combined to describe the key of a song, to simplify our analysis (and for the purpose of demonstration), the CONCAT function is used to combine both the mode value as well as the key value for each record to form a new attribute which is labeled full\_key.

Additionally, AVG operation was used to calculate both the average song and average artist popularities. These were then included as a partition for each record for easy comparison between respective artist/song popularity ratings and the average.

In [ ]:
/* Using CONCAT, SELECT, WHERE, and ORDER BY clauses to combine key and mode columns and select, alias, and filter attributes from joined tables for analysis */

SELECT
    tra.name song_name,
    tra.popularity song_pop,
    AVG(popularity) OVER (
        PARTITION BY album_type) 
        AS avg_song_popularity,
    art.name artist_name,
    artist_popularity,
    AVG(artist_popularity) OVER (
        PARTITION BY album_type) 
        AS avg_artist_popularity,
    followers,
    tempo,
    CONCAT(
        kst.key_let, 
        mst.mode_let
        ) AS full_key,
    duration_ms,
    acousticness,
    danceability,
    energy,
    liveness,
    loudness,
    speechiness,
    valence,
    alb.name album_name,
    release_date,
    album_type

FROM
    MusicProj..trackz tra
    JOIN 
        MusicProj..keystrings kst
        ON tra.[key] = kst.key_num
    JOIN
        MusicProj..modestring mst
        ON tra.mode = mst.mode_num
    LEFT JOIN
        MusicProj..artists art
        ON tra.artists_id = art.id
    LEFT JOIN 
        MusicProj..albums alb
        ON tra.album_id = alb.id
-- Filtering to only show tracks from albums (no compilations or singles)
WHERE
    album_type = 'album'
-- Sorting by highest popularity score
ORDER BY song_pop DESC

### **2.7.1 Understanding Our Data to Identify Outliers**

**(MIN, MAX, COUNT, DISTINCT)**

It's important to understand the scope of our sample size to help identify outliers. In this query, we'll identify the number of years included in the data, the range of years, and the number of distinct artists, albums, and tracks. We'll do this using the MIN and MAX functions to determine the range of years and the COUNT function to determine the number of years, artists, albums, and tracks.

In [ ]:
/* Using MIN, MAX, and COUNT functions to calculate release_date year range as well as number of distinct tracks, artists, and albums) */

SELECT 
    MIN(release_date) as year_range_start, 
    MAX(release_date) as year_range_end,
    COUNT(DISTINCT(release_date)) num_years,
    COUNT(DISTINCT(art.id)) num_artists,
    COUNT(DISTINCT(alb.id)) num_albums,
    COUNT(DISTINCT(tra.id)) num_tracks
FROM
    MusicProj..trackz tra
    JOIN 
        MusicProj..keystrings kst
        ON tra.[key] = kst.key_num
    JOIN
        MusicProj..modestring mst
        ON tra.mode = mst.mode_num
    LEFT JOIN
        MusicProj..artists art
        ON tra.artists_id = art.id
    LEFT JOIN 
        MusicProj..albums alb
        ON tra.album_id = alb.id
WHERE
    album_type = 'album'

### **2.7.2 Finding Outliers**

When looking at year range, we notice that the samples starts in 1886 and ends in 2019. That range does not line up with the total number of years included in the sample (which we calculated as 85). Something's off and, given the context of the data, we suspect it's the album released in the year 1886. This hypothesis is further supported by additional research on the web which tells us that the first album ever recorded was released in 1889. 

Using the WHERE clause, we learn that the album listed with an 1886 release year is actually a soundtrack for a video game called in "The Order: 1886". A quick web search shows us that the album was actually released in 2015.

In [ ]:
/* Using WHERE clause to drill down on record with release year of 1886 */

SELECT
    tra.name song_name,
    art.name artist_name,
    alb.name album_name,
    alb.id,
    release_date

FROM
    MusicProj..trackz tra
    JOIN 
        MusicProj..keystrings kst
        ON tra.[key] = kst.key_num
    JOIN
        MusicProj..modestring mst
        ON tra.mode = mst.mode_num
    LEFT JOIN
        MusicProj..artists art
        ON tra.artists_id = art.id
    LEFT JOIN 
        MusicProj..albums alb
        ON tra.album_id = alb.id
WHERE
    release_date = 1886



### **2.7.3 Correcting Outliers**

Now that we understand the error and have the correct information, we need to correct it. We can do this by updating the record in the albums table with the correct year of 2015. 

Afterwards, we'll select the record to verify that the change was applied.

In [ ]:
/* Updating record to change release_date from 1886 to 2015 */

UPDATE 
    MusicProj..albums
SET 
    release_date = 2015
WHERE 
    id = '1IamuMB8i2g2vEJKNv7NFC' AND 
    artist_id = '0kZQLbj8h2SMVZHUJSbfBQ' AND 
    [name] = 'The Order: 1886 (Video Game Soundtrack)'

/* Verifying the change */

SELECT 
    *
FROM 
    MusicProj..albums
WHERE 
    id = '1IamuMB8i2g2vEJKNv7NFC'

### **2.7.4 Let's Make Those Calculations Again**

**(MIN, MAX, COUNT, DISTINCT)**

Now that we've corrected the incorrect data, let's run the query from above one more time to calculate the correct year range and number of years. 

The results make much more sense now. 1923-2019 is the correct range. We also calculate that the data includes 20,183 total artists with 32,203 total albums from those artists which include a grand total of 52,812 tracks.

In [ ]:
/* Using MIN, MAX, and COUNT to calculate information about dataset */

SELECT 
    MIN(release_date) as year_range_start, 
    MAX(release_date) as year_range_end,
    COUNT(DISTINCT(release_date)) num_years,
    COUNT(DISTINCT(art.id)) num_artists,
    COUNT(DISTINCT(alb.id)) num_albums,
    COUNT(DISTINCT(tra.id)) num_tracks
FROM
    MusicProj..trackz tra
    JOIN 
        MusicProj..keystrings kst
        ON tra.[key] = kst.key_num
    JOIN
        MusicProj..modestring mst
        ON tra.mode = mst.mode_num
    LEFT JOIN
        MusicProj..artists art
        ON tra.artists_id = art.id
    LEFT JOIN 
        MusicProj..albums alb
        ON tra.album_id = alb.id
WHERE 
    album_type = 'album'

# Analyze

### **1\. Calculating Specific Rating Frequency**

**(COUNT, GROUP BY)**

We can use the COUNT function in conjunction with the GROUP BY clause to determine how often artists and songs are awarded specific rating values. This can tell us how likely or unlikely it is for a song or artist to be rated as a specific value.

In [ ]:
/* Using COUNT and GROUP BY functions to determine number of artists and songs per their respective ratings */

SELECT 
    popularity song_popularity, 
    COUNT(popularity) AS songs_with_rating,
    artist_popularity,
    COUNT(artist_popularity) AS artists_with_rating
FROM
    MusicProj..trackz tra
    JOIN 
        MusicProj..keystrings kst
        ON tra.[key] = kst.key_num
    JOIN
        MusicProj..modestring mst
        ON tra.mode = mst.mode_num
    LEFT JOIN
        MusicProj..artists art
        ON tra.artists_id = art.id
    LEFT JOIN 
        MusicProj..albums alb
        ON tra.album_id = alb.id
WHERE album_type = 'album'
GROUP BY
    popularity,
    artist_popularity
ORDER BY 1 DESC, 3 DESC

### **2\. Segmenting by Popularity**

**(CASE, CTE, GROUP BY)**

It may be helpful for our analysis to segment artists by popularity. We'll segment artists into 5 different categories based on popularity. These segments will be described as "very low", "low", "middle", "high", and "very high".

We can accomplish this by applying case statements based on artist\_popularity score within a CTE and grouping the results by the new attribute pop\_seg.

In order to sort the resulting table in a logical format (highest to lowest), we can apply another CASE statement in the ORDER BY clause to assign a numeric rank to each of the string values.

We'll do the same for song popularity in the subsequent query.

In [ ]:
/* Using CASE and CTE to segment artists by popularity and determine number of artists per segment */

WITH 
    popCTE AS (
    SELECT DISTINCT(art.[name]), artist_popularity,
        CASE
            WHEN artist_popularity BETWEEN 0 AND 20 THEN 'very low'
            WHEN artist_popularity BETWEEN 21 AND 40 THEN 'low'
            WHEN artist_popularity BETWEEN 41 AND 60 THEN 'middle'
            WHEN artist_popularity BETWEEN 61 AND 80 THEN 'high'
            ELSE 'very high'
        END as artist_pop_seg
    FROM
        MusicProj..trackz tra
        JOIN 
            MusicProj..keystrings kst
            ON tra.[key] = kst.key_num
        JOIN
            MusicProj..modestring mst
            ON tra.mode = mst.mode_num
        LEFT JOIN
            MusicProj..artists art
            ON tra.artists_id = art.id
        LEFT JOIN 
            MusicProj..albums alb
            ON tra.album_id = alb.id
    WHERE
        album_type = 'album'
)

SELECT artist_pop_seg, COUNT(artist_pop_seg) artists_in_segment
FROM popCTE
GROUP BY artist_pop_seg
ORDER BY 
    CASE 
        WHEN artist_pop_seg = 'very low' THEN 5 
        WHEN artist_pop_seg = 'low' THEN 4 
        WHEN artist_pop_seg = 'middle' THEN 3 
        WHEN artist_pop_seg = 'high' THEN 2 
        ELSE 1 
    END 

In [ ]:
/* Running same CASE statement and CTE as above for trackz table */

WITH 
    pop2CTE AS (
    SELECT DISTINCT(tra.[name]), popularity,
        CASE
            WHEN popularity BETWEEN 0 AND 20 THEN 'very low'
            WHEN popularity BETWEEN 21 AND 40 THEN 'low'
            WHEN popularity BETWEEN 41 AND 60 THEN 'middle'
            WHEN popularity BETWEEN 61 AND 80 THEN 'high'
            ELSE 'very high'
        END as song_pop_seg
    FROM
        MusicProj..trackz tra
        JOIN 
            MusicProj..keystrings kst
            ON tra.[key] = kst.key_num
        JOIN
            MusicProj..modestring mst
            ON tra.mode = mst.mode_num
        LEFT JOIN
            MusicProj..artists art
            ON tra.artists_id = art.id
        LEFT JOIN 
            MusicProj..albums alb
            ON tra.album_id = alb.id
    WHERE
        album_type = 'album'
)

SELECT song_pop_seg, COUNT(song_pop_seg) songs_in_segment
FROM pop2CTE
GROUP BY song_pop_seg
ORDER BY 
    CASE 
        WHEN song_pop_seg = 'very low' THEN 5 
        WHEN song_pop_seg = 'low' THEN 4 
        WHEN song_pop_seg = 'middle' THEN 3 
        WHEN song_pop_seg = 'high' THEN 2 
        ELSE 1 
    END

### **3.1 Determining Commonality of Keys/Modes**

The most and least frequently used keys are calculated using a CTE containing the CONCAT function for key name which was then used to run a COUNT operation to determine number of records per key. 

From this query, we determine that the most commonly used key is Cmaj while the least common key for a song in our sample is Ebmin.

In [ ]:
/* Creating CTE to determine most and least common keys */

WITH 
    MusicCTE AS (
SELECT
    tra.name song_name,
    tra.popularity song_pop,
    art.name artist_name,
    artist_popularity,
    followers,
    tempo,
    kst.key_let,
    mst.mode_let,
    CONCAT(kst.key_let, mst.mode_let) AS full_key,
    duration_ms,
    acousticness,
    danceability,
    energy,
    liveness,
    loudness,
    speechiness,
    valence,
    alb.name album_name,
    release_date,
    album_type

FROM
    MusicProj..trackz tra
    JOIN 
        MusicProj..keystrings kst
        ON tra.[key] = kst.key_num
    JOIN
        MusicProj..modestring mst
        ON tra.mode = mst.mode_num
    LEFT JOIN
        MusicProj..artists art
        ON tra.artists_id = art.id
    LEFT JOIN 
        MusicProj..albums alb
        ON tra.album_id = alb.id
WHERE
    album_type = 'album'
)

-- Determining most common key for songs (CMaj is most common, Ebmin is least common)

SELECT 
    full_key, 
    COUNT(full_key) as songs_per_full_key
FROM 
    MusicCTE
GROUP BY 
    full_key
ORDER BY 
    COUNT(full_key) DESC

### **3.2 Commonalities of Key/Mode cont'd**

**(COUNT, GROUP BY, ORDER BY)**

We'll do the same as above for individual alphabetic keys and again for individual mode. This time, since we won't be using a the CONCAT function, we can run our query without a CTE.

In [ ]:
/* Using COUNT function to determine songs per individual alphabetic key */

SELECT
    key_let,
    COUNT(key_let) as songs_per_key
FROM
    MusicProj..trackz tra
    JOIN 
        MusicProj..keystrings kst
        ON tra.[key] = kst.key_num
    JOIN
        MusicProj..modestring mst
        ON tra.mode = mst.mode_num
    LEFT JOIN
        MusicProj..artists art
        ON tra.artists_id = art.id
    LEFT JOIN 
        MusicProj..albums alb
        ON tra.album_id = alb.id
WHERE
    album_type = 'album'
GROUP BY
    key_let
ORDER BY songs_per_key DESC


In [ ]:
-- Using same query as above to determine songs per mode

SELECT
    mode_let,
    COUNT(mode_let) as songs_per_mode
FROM
    MusicProj..trackz tra
    JOIN 
        MusicProj..keystrings kst
        ON tra.[key] = kst.key_num
    JOIN
        MusicProj..modestring mst
        ON tra.mode = mst.mode_num
    LEFT JOIN
        MusicProj..artists art
        ON tra.artists_id = art.id
    LEFT JOIN 
        MusicProj..albums alb
        ON tra.album_id = alb.id
WHERE
    album_type = 'album'
GROUP BY
    mode_let
ORDER BY songs_per_mode DESC

### **4\. Calculating AVG Song Popularity**

**(AVG)**

Average track popularity was calculated using AVG function as a benchmark for individual track popularity.

In [ ]:
/* Using AVG to find average track popularity score */

SELECT 
    AVG(tra.popularity) as avg_track_pop
FROM
    MusicProj..trackz tra
    LEFT JOIN 
        MusicProj..artists art
        ON tra.artists_id = art.id
    LEFT JOIN 
        MusicProj..albums alb
        ON tra.album_id = alb.id
WHERE
    album_type = 'album'

### **5\. Finding Each Artist's Average Song Popularity**

**(AVG)**

  

Next, we'll use the AVG function and GROUP BY clause to determine each artist's average song popularity score.

In [ ]:
/* Using AVG function to determine average song popularity score by artist */

SELECT art.name, AVG(popularity) as avg_song_pop
FROM
    MusicProj..trackz tra
    JOIN 
        MusicProj..keystrings kst
        ON tra.[key] = kst.key_num
    JOIN
        MusicProj..modestring mst
        ON tra.mode = mst.mode_num
    LEFT JOIN
        MusicProj..artists art
        ON tra.artists_id = art.id
    LEFT JOIN 
        MusicProj..albums alb
        ON tra.album_id = alb.id
WHERE album_type = 'album'
GROUP BY art.name
ORDER BY avg_song_pop desc

### **6\. Comparing Artist's Average Song Popularity to Total Average**

**(CAST, CTE)**

Table including total average song and artist popularity as well as each artist's deviation from total artist popularity average is saved as view for importing to Tableau.

Additionally, values were cast as decimals to give clearer view of numbers.

In [ ]:
/* Using CTE and CAST to determine how individual artist popularity and song popularity compares to respective averages */

--Creating CTE including avg popularity calculations and combined key/mode attributes

USE MusicProj

GO

CREATE VIEW ArtistPopularity as

WITH 
    SpotifyCTE (
    song_name,
    song_pop,
    avg_song_popularity,
    artist_name,
    artist_popularity,
    avg_artist_popularity,
    followers,
    tempo,
    full_key,
    duration_ms,
    acousticness,
    danceability,
    energy,
    liveness,
    loudness,
    speechiness,
    valence,
    album_name,
    release_date,
    album_type
    )
AS
(
SELECT
    tra.name song_name,
    tra.popularity song_pop,
    AVG(popularity) OVER (
        PARTITION BY album_type) 
        AS avg_song_popularity,
    art.name artist_name,
    artist_popularity,
    AVG(artist_popularity) OVER (
        PARTITION BY album_type) 
        AS avg_artist_popularity,
    followers,
    tempo,
    CONCAT(
        kst.key_let, 
        mst.mode_let
        ) AS full_key,
    duration_ms,
    acousticness,
    danceability,
    energy,
    liveness,
    loudness,
    speechiness,
    valence,
    alb.name album_name,
    release_date,
    album_type

FROM
    MusicProj..trackz tra
    JOIN 
        MusicProj..keystrings kst
        ON tra.[key] = kst.key_num
    JOIN
        MusicProj..modestring mst
        ON tra.mode = mst.mode_num
    LEFT JOIN
        MusicProj..artists art
        ON tra.artists_id = art.id
    LEFT JOIN 
        MusicProj..albums alb
        ON tra.album_id = alb.id
WHERE
    album_type = 'album'
)

-- Determining artist popularity deviation from average 

SELECT 
    artist_name, 
    artist_popularity, 
    avg_artist_popularity,
    (cast(artist_popularity as decimal)-cast(avg_artist_popularity as decimal)) as art_dev
FROM 
    SpotifyCTE
GROUP BY 
    artist_name, artist_popularity, avg_artist_popularity


# Share

## 1\. Saving Tables in Views

We'll be creating our visualizations for this data in Tableau. In order to export/import from Azure Data Studio into Tableau more easily, we'll save several tables as views to use in our visualizations.

In [ ]:
/* Saving table as View */

USE MusicProj

GO

CREATE VIEW dbo.FullTrackTable as
SELECT
    tra.name song_name,
    tra.popularity song_pop,
    AVG(popularity) OVER (
        PARTITION BY album_type) 
        AS avg_song_popularity,
    art.name artist_name,
    artist_popularity,
    AVG(artist_popularity) OVER (
        PARTITION BY album_type) 
        AS avg_artist_popularity,
    followers,
    tempo,
    CONCAT(
        kst.key_let, 
        mst.mode_let
        ) AS full_key,
    duration_ms,
    acousticness,
    danceability,
    energy,
    liveness,
    loudness,
    speechiness,
    valence,
    alb.name album_name,
    release_date,
    album_type

FROM
    MusicProj..trackz tra
    JOIN 
        MusicProj..keystrings kst
        ON tra.[key] = kst.key_num
    JOIN
        MusicProj..modestring mst
        ON tra.mode = mst.mode_num
    LEFT JOIN
        MusicProj..artists art
        ON tra.artists_id = art.id
    LEFT JOIN 
        MusicProj..albums alb
        ON tra.album_id = alb.id
WHERE
    album_type = 'album'


In [ ]:
/* Saving previous KeyCommonality query as view */

USE MusicProj

GO

CREATE VIEW dbo.KeyCommon as

WITH keyCTE (
    full_key
)
AS
(
SELECT
    CONCAT(kst.key_let, mst.mode_let) AS full_key
FROM
    MusicProj..trackz tra
    JOIN 
        MusicProj..keystrings kst
        ON tra.[key] = kst.key_num
    JOIN
        MusicProj..modestring mst
        ON tra.mode = mst.mode_num
    LEFT JOIN
        MusicProj..artists art
        ON tra.artists_id = art.id
    LEFT JOIN 
        MusicProj..albums alb
        ON tra.album_id = alb.id
WHERE
    album_type = 'album'
)

SELECT full_key, COUNT(full_key) as key_count
FROM keyCTE
GROUP BY full_key

In [ ]:
USE MusicProj

GO

CREATE VIEW dbo.SongSegment2 as

WITH 
    pop21CTE AS (
    SELECT DISTINCT(tra.[name]) track_name, art.[name] artist_name, alb.[name] album_name, artist_popularity, duration_ms, key_let, mode_let, danceability, tempo, energy, followers, liveness, loudness, release_date, speechiness, valence, popularity,
        CASE
            WHEN popularity BETWEEN 0 AND 20 THEN 'very low'
            WHEN popularity BETWEEN 21 AND 40 THEN 'low'
            WHEN popularity BETWEEN 41 AND 60 THEN 'middle'
            WHEN popularity BETWEEN 61 AND 80 THEN 'high'
            WHEN popularity BETWEEN 81 AND 100 THEN 'very high'
        END as song_pop_seg
    FROM
        MusicProj..trackz tra
        JOIN 
            MusicProj..keystrings kst
            ON tra.[key] = kst.key_num
        JOIN
            MusicProj..modestring mst
            ON tra.mode = mst.mode_num
        LEFT JOIN
            MusicProj..artists art
            ON tra.artists_id = art.id
        LEFT JOIN 
            MusicProj..albums alb
            ON tra.album_id = alb.id
    WHERE
        album_type = 'album'
)

SELECT track_name, artist_name, album_name, popularity, artist_popularity, key_let, mode_let, duration_ms, danceability, tempo, energy, followers, liveness, loudness, release_date, speechiness, valence, song_pop_seg
FROM pop21CTE
GROUP BY track_name, artist_name, album_name, song_pop_seg, popularity, artist_popularity, key_let, mode_let, duration_ms, danceability, tempo, energy, followers, liveness, loudness, release_date, speechiness, valence
/*ORDER BY 
    CASE 
        WHEN song_pop_seg = 'very low' THEN 5 
        WHEN song_pop_seg = 'low' THEN 4 
        WHEN song_pop_seg = 'middle' THEN 3 
        WHEN song_pop_seg = 'high' THEN 2 
        ELSE 1 
    END*/

In [ ]:
USE MusicProj

GO

CREATE VIEW dbo.SongAttributes as

SELECT
    tra.name song_name,
    tra.popularity song_pop,
    art.name artist_name,
    artist_popularity,
    followers,
    tempo,
    kst.key_let, 
    mst.mode_let,
    duration_ms,
    acousticness,
    danceability,
    energy,
    liveness,
    loudness,
    speechiness,
    valence,
    alb.name album_name,
    release_date,
    album_type

FROM
    MusicProj..trackz tra
    JOIN 
        MusicProj..keystrings kst
        ON tra.[key] = kst.key_num
    JOIN
        MusicProj..modestring mst
        ON tra.mode = mst.mode_num
    LEFT JOIN
        MusicProj..artists art
        ON tra.artists_id = art.id
    LEFT JOIN 
        MusicProj..albums alb
        ON tra.album_id = alb.id
WHERE
    album_type = 'album'


In [ ]:
/* Creating view for SongPopularity deviation table */

USE MusicProj

GO

CREATE VIEW SongPopularity as

WITH 
    SpotifyCTE (
    song_name,
    song_pop,
    avg_song_popularity,
    artist_name,
    artist_popularity,
    avg_artist_popularity,
    followers,
    tempo,
    full_key,
    duration_ms,
    acousticness,
    danceability,
    energy,
    liveness,
    loudness,
    speechiness,
    valence,
    album_name,
    release_date,
    album_type
    )
AS
(
SELECT
    tra.name song_name,
    tra.popularity song_pop,
    AVG(popularity) OVER (
        PARTITION BY album_type) 
        AS avg_song_popularity,
    art.name artist_name,
    artist_popularity,
    AVG(artist_popularity) OVER (
        PARTITION BY album_type) 
        AS avg_artist_popularity,
    followers,
    tempo,
    CONCAT(
        kst.key_let, 
        mst.mode_let
        ) AS full_key,
    duration_ms,
    acousticness,
    danceability,
    energy,
    liveness,
    loudness,
    speechiness,
    valence,
    alb.name album_name,
    release_date,
    album_type

FROM
    MusicProj..trackz tra
    JOIN 
        MusicProj..keystrings kst
        ON tra.[key] = kst.key_num
    JOIN
        MusicProj..modestring mst
        ON tra.mode = mst.mode_num
    LEFT JOIN
        MusicProj..artists art
        ON tra.artists_id = art.id
    LEFT JOIN 
        MusicProj..albums alb
        ON tra.album_id = alb.id
WHERE
    album_type = 'album'
)

-- Determining song popularity deviation from average

SELECT 
    song_name, 
    artist_name, 
    song_pop,
    avg_song_popularity,
    (cast(song_pop as decimal)-cast(avg_song_popularity as decimal)) as song_dev
FROM 
    SpotifyCTE
GROUP BY 
    song_name, artist_name, song_pop, avg_song_popularity